Below is the answer for solving code problems. 

### exercise 1: 
 why is the returned result wrong?<br>
 The issue in your code is that you are comparing the fruit_id with the loop index idx, which does not represent the index of the fruit in the set. 

In [2]:
#How could this be fixed?
from typing import Set

def id_to_fruit(fruit_id: int, fruits: Set[str]) -> str:
    fruit_list = list(fruits)
    
    if fruit_id < 0 or fruit_id >= len(fruit_list):
        raise RuntimeError(f"Fruit with id {fruit_id} does not exist")

    return fruit_list[fruit_id]


# exercise 2:
Can you spot the obvious error?< <br>
It seems there is a mistake in the assignment of values to the different columns

In [ ]:
import numpy as np
def swap(coords: np.ndarray):
    coords[:, 0], coords[:, 1], coords[:, 2], coords[:, 3] = coords[:, 1].copy(), coords[:, 0].copy(), coords[:, 3].copy(), coords[:, 2].copy()
    return coords


# exercies 3:

the 'pandas' library can be used to simplify the process of reading the CSV file and working with the data.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_data(csv_file_path: str):
    # Read data using pandas
    df = pd.read_csv(csv_file_path)

    # Extract precision and recall columns
    precision = df['precision'].values
    recall = df['recall'].values

    plt.plot(recall, precision)
    plt.ylim([-0.05, 1.05])
    plt.xlim([-0.05, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()

ModuleNotFoundError: No module named 'pandas'

# exerecise 4: <br>
The issue arises from the mismatch in the size of the labels provided to the loss function when training the discriminator. This happens because the generator generates samples with a different batch size than the real samples.

To fix this, you should adjust the size of the generated samples labels to match the batch size of the generator <br>
As for the cosmetic bug, the code contains unnecessary try-except blocks and duplicated data loading for the MNIST dataset.

In [3]:
import torch
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchvision
NoneType = type(None)
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from PIL import Image
import torchvision.transforms.functional as TF
from torchvision.models import vgg11
from torchvision.models import mobilenet_v2
import torchvision.transforms as transforms
import time

def train_gan(batch_size: int = 32, num_epochs: int = 100, device: str = "cuda:0" if torch.cuda.is_available() else "cpu"):

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    # Download MNIST dataset
    torchvision.datasets.MNIST.resources = [
        ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
    ]

    train_set = torchvision.datasets.MNIST(root=".", train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

    # example data
    real_samples, mnist_labels = next(iter(train_loader))

    fig = plt.figure()
    for i in range(16):
        sub = fig.add_subplot(4, 4, 1 + i)
        sub.imshow(real_samples[i].reshape(28, 28), cmap="gray_r")
        sub.axis('off')

    fig.tight_layout()
    fig.suptitle("Real images")
    display(fig)

    time.sleep(5)

    # Set up training
    discriminator = Discriminator().to(device)
    generator = Generator().to(device)
    lr = 0.0001
    loss_function = nn.BCELoss()
    optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
    optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

    # train
    for epoch in range(num_epochs):
        for n, (real_samples, mnist_labels) in enumerate(train_loader):

            # Data for training the discriminator
            real_samples = real_samples.to(device=device)
            real_samples_labels = torch.ones((batch_size, 1)).to(device=device)
            latent_space_samples = torch.randn((batch_size, 100)).to(device=device)
            generated_samples = generator(latent_space_samples)
            generated_samples_labels = torch.zeros((batch_size, 1)).to(device=device)
            all_samples = torch.cat((real_samples, generated_samples))
            all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))

            # Training the discriminator
            discriminator.zero_grad()
            output_discriminator = discriminator(all_samples)
            loss_discriminator = loss_function(output_discriminator, all_samples_labels)
            loss_discriminator.backward()
            optimizer_discriminator.step()

            # Data for training the generator
            latent_space_samples = torch.randn((batch_size, 100)).to(device=device)

            # Training the generator
            generator.zero_grad()
            generated_samples = generator(latent_space_samples)
            output_discriminator_generated = discriminator(generated_samples)
            # Adjust the size of generated_samples_labels to match the batch size of the generator
            generated_samples_labels = torch.ones((batch_size, 1)).to(device=device)
            loss_generator = loss_function(output_discriminator_generated, generated_samples_labels)
            loss_generator.backward()
            optimizer_generator.step()
            # Show loss and samples generated
            if n == batch_size - 1:
                name = f"Generate images\n Epoch: {epoch} Loss D.: {loss_discriminator:.2f} Loss G.: {loss_generator:.2f}"
                generated_samples = generated_samples.detach().cpu().numpy()
                fig = plt.figure()
                for i in range(16):
                    sub = fig.add_subplot(4, 4, 1 + i)
                    sub.imshow(generated_samples[i].reshape(28, 28), cmap="gray_r")
                    sub.axis('off')
                fig.suptitle(name)
                fig.tight_layout()
                clear_output(wait=False)
                display(fig)



ModuleNotFoundError: No module named 'torch'